In [1]:
import numpy as np
from astropy.io import ascii
from astropy.table import Table
import sys
import matplotlib.pyplot as plt

In [2]:
#Cardelli Extinction for spectra:

def Cardelli(wave,flux,Av,Rv):
    '''Will de-redden an input spectrum based on the Cardelli law:
    
    Parameters:
    wave (angstrom): wavelengths of input spectrum in Angstrom
    flux (erg/s/cm/Ang): flux of spectrum; scaled version of this flux are fine.
    Av:
    Rv:
    
    Returns: a new flux array in same units as input
    '''

    wave_micon=wave*1.0e-4
    wave_inverse=1.0/wave_micon
    
    A_lambda=np.zeros(len(wave))
    ax=np.zeros(len(wave))
    bx=np.zeros(len(wave))
    
    for i in range(len(wave)):
        if((wave_inverse[i] > 0.3) and (wave_inverse[i] < 1.1)):
            ax[i]=0.574*wave_inverse[i]**1.61
            bx[i]=-0.527*wave_inverse[i]**1.61
        if((wave_inverse[i] > 1.1) and (wave_inverse[i] < 3.3)):
            y=wave_inverse[i] - 1.82
            ax[i]=1+ 0.17699*y - 0.50447*y**2. - 0.02427*y**3.+0.72085*y**4.+0.01979*y**5.-0.77530*y**6.+0.32999*y**7.
            bx[i]=1.41338*y+2.28305*y**2.+1.07233*y**3.-5.38434*y**4.-0.62251*y**5.+5.30260*y**6.-2.09002*y**7.
        if((wave_inverse[i] > 3.3) and (wave_inverse[i] < 8.0)):
            if(wave_inverse[i] > 5.9):
                Fa=-0.04473*(wave_inverse[i] - 5.9)**2.-0.009779*(wave_inverse[i]-5.9)**3.
                Fb=0.2130*(wave_inverse[i] - 5.9)**2. +0.1207*(wave_inverse[i] -5.9)**3.
            else:
                Fa = 0.0
                Fb = 0.0

            ax[i]=1.752 - 0.316*wave_inverse[i]-0.104/((wave_inverse[i]-4.67)**2.+0.341) +Fa
            bx[i]=-3.090+1.825*wave_inverse[i]+1.206/((wave_inverse[i] - 4.62)**2.+0.263) +Fb

    A_lambda=(ax+bx/Rv)*Av
    #print(A_lambda)
    NewSpec=flux*10.**(0.4*A_lambda)
    results = NewSpec
    return results

## Read in file, correct, write out.

In [3]:
file='spectra.csv'
data = ascii.read(file,format='csv')
#first epoch SN
l = data['lambda(A)']
flux = data['flux']
fluxerr = data['fluxerror']
#second epoch SN
lg = data['lambdag(A)']
fluxg = data['fluxg']
#host galaxy spec
lga = data['lambda2']
fluxga = data['flux2']

In [1]:
#data = ascii.read('PS1-11aop-host-comb-20140923-dered.flm')
#data
data = ascii.read('/Users/adaezeibik/Downloads/PS1-11aop-host-comb-20140923-dered.flm')

AttributeError: 'builtin_function_or_method' object has no attribute 'read'

In [9]:
0.0347*3.1

0.10757000000000001

## Use  https://irsa.ipac.caltech.edu/applications/DUST/ to estimate the EBV of the source
## using the source RA and DEC

In [3]:
#data = ascii.read('PS1-11aop-host-comb-20140923.flm',names=['waves','flux'])

EBmV = 0.0347
Rv = 3.1
Av = EBmV*Rv

new_flux = Cardelli(lga,fluxga,Av,Rv)

outfile = 'PS1-11aop-host-dered.fits'
data2 = [lga,new_flux]
ascii.write(data2,outfile)

NameError: name 'lga' is not defined

In [13]:
new_flux = Cardelli(np.array([6562.819]),np.array([1.342e-15]),Av,Rv) #6562.819

new_flux


array([1.45525708e-15])

In [6]:
data2

waves,flux
float64,float64
3801.0,0.1050628423690796
3802.91976329028,0.08556422591209412
3804.83952658056,0.1372953653335571
3806.75928987084,0.1593190580606461
3808.67905316112,0.07919652760028839
3810.59881645141,-0.01335764117538929
3812.51857974169,-0.1231140941381454
3814.43834303197,-0.09061825275421143
3816.35810632225,0.07177010178565979


In [3]:
#Do the same for the explosion spectra:

In [4]:
data = ascii.read('Explosion/mmtblue-20120720-h300140-o1.txt',names=['waves','flux'])

EBmV = 0.0347
Rv = 3.1
Av = EBmV*Rv

new_flux = Cardelli(data['waves'],data['flux'],Av,Rv)

outfile = 'Explosion/mmtblue-20120720-h300140-o1-dered.flm'
data2 = [data['waves'],new_flux]
ascii.write(data2,outfile)

In [5]:
data = ascii.read('Explosion/checto015.2011mHm300140-20111027.100+error.flm',names=['waves','flux','dflux'])

EBmV = 0.0347
Rv = 3.1
Av = EBmV*Rv

new_flux = Cardelli(data['waves'],data['flux'],Av,Rv)

outfile = 'Explosion/checto015.2011mHm300140-20111027.100+error-dered.flm'
data2 = [data['waves'],new_flux]
ascii.write(data2,outfile)

In [13]:
# Extinction correction and rest frame correction for C64 Host galaxy Gemini spectra

data = ascii.read('/Users/adaezeibik/virtual_ubuntu/Q2-1/Q2-2/corr_skysub_version/c64_phv2_comb.csv',names=['waves','flux'])

EBmV = 0.0078 
Rv = 3.1
Av = EBmV*Rv
z = 0.122
new_flux = Cardelli(data['waves'],data['flux'],Av,Rv)

outfile = 'c64_phv2_dered.csv'
data2 = [data['waves']/(1+z),new_flux]
ascii.write(data2,outfile)

In [17]:
#Extinction correction for C27 host galaxy spectra
#flux = [halpha ,hbeta,nitrogen,oxygen]
wave = np.array([6562.819,4861.333,6583.460,5006.843])
flux = np.array([5.808e-15, 8.311e-16, 1.9912e-15, 1.188e-16]) #C27 host galaxy spectra from sdss-iii
EBmV = 0.0197 
Rv = 3.1
Av = EBmV*Rv

new_flux = Cardelli(wave,flux,Av,Rv)
new_flux

array([6.08565624e-15, 8.88228053e-16, 2.08601168e-15, 1.26648229e-16])

In [18]:
#Extinction correction for R49 host galaxy spectra
#flux = [halpha ,hbeta,nitrogen,oxygen]
wave = np.array([6562.819,4861.333,6583.460,5006.843])
flux = np.array([7.69869e-15, 9.9047e-16, 3.72269e-15, 2.5137e-15]) #R49 host galaxy spectra from lamost
EBmV = 0.0483
Rv = 3.1
Av = EBmV*Rv

new_flux = Cardelli(wave,flux,Av,Rv)
new_flux

array([8.62985326e-15, 1.16528302e-15, 4.17109806e-15, 2.93928771e-15])

In [7]:
# Extinction correction and rest frame correction for C64 prs Host galaxy Gemini spectra
#Delete the header info after converting the file (if not it will not read in)
data = ascii.read('/Users/adaezeibik/virtual_ubuntu/Q2-1/Q2-1/c64prs-combine.csv',names=['waves','flux'])

EBmV = 0.0078 #from SFD
Rv = 3.1
Av = EBmV*Rv
z = 0.2673 # from spec
new_flux = Cardelli(data['waves'],data['flux'],Av,Rv)

outfile = 'c64prs-dered.csv'
data2 = [data['waves']/(1+z),new_flux]
ascii.write(data2,outfile)

OSError: File c64prs-dered.csv already exists. If you mean to replace it then use the argument "overwrite=True".

In [6]:
data = ascii.read('/Users/adaezeibik/virtual_ubuntu/R18-spec/new-ext/r18prs-clip.csv',names=['waves','flux'])
EBmV = 0.0729 #from SFD 0.0729 +/-  0.0040
Rv = 3.1
Av = EBmV*Rv
z = 0.12817 # from spec
new_flux = Cardelli(data['waves'],data['flux'],Av,Rv)
outfile = '/Users/adaezeibik/virtual_ubuntu/R18-spec/new-ext/r18prs-dered.csv'
data2 = [data['waves']/(1+z),new_flux]
ascii.write(data2,outfile)

In [8]:
#using pypeit product
data = ascii.read('/Users/adaezeibik/virtual_ubuntu/example_r18_field/r18prs.flm',names=['waves','flux'])
EBmV = 0.0729 #from SFD 0.0729 +/-  0.0040
Rv = 3.1
Av = EBmV*Rv
z = 0.12817 # from spec
new_flux = Cardelli(data['waves'],data['flux'],Av,Rv)
outfile = '/Users/adaezeibik/virtual_ubuntu/example_r18_field/pypeit-r18prs-dered.csv'
data2 = [data['waves']/(1+z),new_flux]
ascii.write(data2,outfile)

In [9]:
#using pypeit product, no redshift , just corrected for extinction
data = ascii.read('/Users/adaezeibik/virtual_ubuntu/example_r16_field/r16prs.flm',names=['waves','flux'])
EBmV = 0.0495 #from SFD 0.0495 +/-  0.0003
Rv = 3.1
Av = EBmV*Rv
new_flux = Cardelli(data['waves'],data['flux'],Av,Rv)
outfile = '/Users/adaezeibik/virtual_ubuntu/example_r16_field/pypeit-r16prs_extincorr.csv'
data2 = [data['waves'],new_flux]
ascii.write(data2,outfile)

In [3]:
#using pypeit product, no redshift , just corrected for extinction
data = ascii.read('/Users/adaezeibik/virtual_ubuntu/c64-comb/C64prs-comb.flm',names=['waves','flux'])
EBmV = 0.0078 #from SFD
Rv = 3.1
Av = EBmV*Rv
new_flux = Cardelli(data['waves'],data['flux'],Av,Rv)
outfile = '/Users/adaezeibik/virtual_ubuntu/c64-comb/C64prs_comb_extincorr.csv'
data2 = [data['waves'],new_flux]
ascii.write(data2,outfile)